In [1]:
import pandas as pd
import numpy as np
import os

# Function to clean numeric values
def clean_numeric(value):
    try:
        value = str(value).replace('\xa0', '').replace('(', '').replace(')', '').replace(',', '')
        return float(value)
    except ValueError:
        return np.nan

# Function to convert IP notation to real numbers
def convert_ip_to_real(ip):
    if pd.isna(ip):
        return np.nan
    ip_str = str(ip)
    if '.' in ip_str:
        parts = ip_str.split('.')
        whole = int(parts[0])
        fraction = int(parts[1]) if len(parts) > 1 else 0
        if fraction == 1:
            return whole + 1/3
        elif fraction == 2:
            return whole + 2/3
        else:
            return whole
    return float(ip)

# Function to calculate ERA
def calculate_era(df):
    return (df['ER'] * 9) / df['IP_real']

# Function to calculate WHIP
def calculate_whip(df):
    return (df['H'] + df['BB']) / df['IP_real']

# Function to calculate extra base hits against
def calculate_extra_base_hits_against(df):
    return df['2B'] + df['3B'] + df['HR']

# Function to calculate total bases against
def calculate_total_bases_against(df):
    return df['H'] + df['2B'] + 2 * df['3B'] + 3 * df['HR']

# Function to calculate rolling stats
def calculate_rolling_stats(df, window, suffix):
    rolling_df = df.rolling(window=window, min_periods=1).sum()
    rolling_df['ERA'] = calculate_era(rolling_df)
    rolling_df['WHIP'] = calculate_whip(rolling_df)
    rolling_df['XB_against'] = calculate_extra_base_hits_against(rolling_df)
    rolling_df['TB_against'] = calculate_total_bases_against(rolling_df)
    rolling_df = rolling_df[['IP_real', 'H', 'BF', 'HR', 'R', 'ER', 'BB', 'SO', 'XB_against', 'TB_against', 'ERA', 'WHIP']]
    rolling_df.columns = [f'{col}_{suffix}' for col in rolling_df.columns]
    return rolling_df.round(3)

# Load pitcher IDs
idlist = pd.read_csv('pitcher_ids.csv')
pitcher_ids = idlist.key_bbref

# Load game PKs (if needed)
game_pks = pd.read_csv('game_pks.csv')

for id in pitcher_ids:

    if not id or pd.isna(id):
        continue

    file_path = f'pitchers/{id}_pitching.csv' 
    
    if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
        try:
            df = pd.read_csv(file_path)
        except pd.errors.EmptyDataError:
            print(f"File for ID {id} is empty.")
            continue
        except pd.errors.ParserError:
            print(f"File for ID {id} is improperly formatted.")
            continue
    else:
        print(f"File for ID {id} does not exist or is empty.")
        continue

    # Remove the irrelevant column 'Gtm'
    df = df.drop(columns=['Gtm'])

    # Ensure the 'game_date' column is in datetime format
    df['game_date'] = pd.to_datetime(df['game_date'])

    # Extract the year from the 'game_date' column
    df['season'] = df['game_date'].dt.year

    # Define columns to convert to numeric
    numeric_columns = ['IP', 'H', 'R', 'ER', 'BB', 'SO', 'HR', 'BF', '2B', '3B', 'IBB']

    # Apply cleaning function to numeric columns
    for col in numeric_columns:
        df[col] = df[col].apply(clean_numeric)

    # Fill NaN values with 0 for numerical calculations
    df[numeric_columns] = df[numeric_columns].fillna(0)

    # Ensure columns are of correct numeric type
    df[numeric_columns] = df[numeric_columns].astype(float)

    # Create the IP_real column
    df['IP_real'] = df['IP'].apply(convert_ip_to_real)

    # Exclude non-numeric columns from rolling stats calculation
    rolling_df = df[numeric_columns + ['IP_real']].copy()

    # Calculate rolling stats for the last 20 games and shift by one row
    rolling_stats_20 = calculate_rolling_stats(rolling_df, 20, '20').shift(1).fillna(0)

    # Calculate rolling stats for the last 20 games and shift by one row
    rolling_stats_10 = calculate_rolling_stats(rolling_df, 10, '10').shift(1).fillna(0)

    # Calculate rolling stats for the last 5 games and shift by one row
    rolling_stats_5 = calculate_rolling_stats(rolling_df, 5, '5').shift(1).fillna(0)

    # Calculate rolling stats for the last 20 games and shift by one row
    rolling_stats_3 = calculate_rolling_stats(rolling_df, 3, '3').shift(1).fillna(0)

    # Calculate season-long stats for each year and shift by one row
    season_stats = pd.DataFrame()
    for year in df['season'].unique():
        season_df = df[df['season'] == year][numeric_columns + ['IP_real']].copy()
        season_cumsum = season_df.cumsum().shift(1).fillna(0)
        season_cumsum['ERA'] = calculate_era(season_cumsum)
        season_cumsum['WHIP'] = calculate_whip(season_cumsum)
        season_cumsum['XB_against'] = calculate_extra_base_hits_against(season_cumsum)
        season_cumsum['TB_against'] = calculate_total_bases_against(season_cumsum)
        season_cumsum = season_cumsum[['IP_real', 'H', 'BF', 'HR', 'R', 'ER', 'BB', 'SO', 'XB_against', 'TB_against', 'ERA', 'WHIP']]
        season_cumsum.columns = [f'{col}_current' for col in season_cumsum.columns]
        season_stats = pd.concat([season_stats, season_cumsum])

    # Ensure the season_stats index aligns with the original dataframe
    season_stats.index = df.index

    # Combine all the stats into a single dataframe
    final_df = pd.concat([df, rolling_stats_20, rolling_stats_10, rolling_stats_5, rolling_stats_3, season_stats], axis=1)

    # Round the combined dataframe stats to 3 decimal points
    final_df = final_df.round(3)

    # Display the combined dataframe
    print(final_df.tail())

    # Save the combined stats to a CSV file
    final_df.to_csv(f'pitchers/{id}_stats_pitching.csv', index=False)

    print(f"Generated stats for {id} and saved to CSV file.")

    Rk  Gcar        Date   Tm Unnamed: 5  Opp    Rslt    Inngs     Dec  DR  \
17   2   402  2023-05-19  COL          @  TEX   L,2-7  8-GF(8)     NaN   1   
18   3   403  2023-05-20  COL          @  TEX  L,5-11      5-6     NaN   0   
19   4   404  2023-07-04  COL          @  HOU   L,1-4  8-GF(8)     NaN  44   
20   5   405  2023-07-08  COL          @  SFG   L,3-5      6-7     NaN   3   
21   6   406  2023-07-18  COL        NaN  HOU   W,4-3      4-5  W(1-0)   9   

    ...  BF_current  HR_current  R_current  ER_current  BB_current  \
17  ...         3.0         0.0        0.0         0.0         0.0   
18  ...         7.0         0.0        0.0         0.0         1.0   
19  ...        15.0         2.0        3.0         3.0         2.0   
20  ...        21.0         2.0        3.0         3.0         2.0   
21  ...        26.0         2.0        3.0         3.0         2.0   

    SO_current  XB_against_current  TB_against_current  ERA_current  \
17         0.0                 1.0     